<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo\">


# Parallel and Distributed Machine Learning

[Dask-ML](https://dask-ml.readthedocs.io) has resources for parallel and distributed machine learning.


## Types of Scaling

There are a couple of distinct scaling problems you might face.
The scaling strategy depends on which problem you're facing.

1. CPU-Bound: Data fits in RAM, but training takes too long. Many hyperparameter combinations, a large ensemble of many models, etc.
2. Memory-bound: Data is larger than RAM, and sampling isn't an option.

* For in-memory problems, just use scikit-learn (or your favorite ML library).
* For large models, use `dask_ml.joblib` and your favorite scikit-learn estimator
* For large datasets, use `dask_ml` estimators

![](images/ml-dimensions.png)

## Scikit-Learn in 5 Minutes

Scikit-Learn has a nice, consistent API.

1. You instantiate an `Estimator` (e.g. `LinearRegression`, `RandomForestClassifier`, etc.). All of the models *hyperparameters* (user-specified parameters, not the ones learned by the estimator) are passed to the estimator when it's created.
2. You call `estimator.fit(X, y)` to train the estimator.
3. Use `estimator` to inspect attributes, make predictions, etc. 

Let's generate some random data.

In [ ]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=10000, n_features=4, random_state=0)
X[:8]

In [ ]:
y[:8]

We'll fit a Support Vector Classifier.

In [ ]:
from sklearn.svm import SVC

Create the estimator and fit it.

In [ ]:
estimator = SVC(random_state=0)
estimator.fit(X, y)

Inspect the learned attributes.

In [ ]:
estimator.support_vectors_[:4]

Check the accuracy.

In [ ]:
estimator.score(X, y)

## Hyperparameters

Most models have *hyperparameters*. They affect the fit, but are specified up front instead of learned during training.

In [ ]:
estimator = SVC(C=0.00001, shrinking=False, random_state=0)
estimator.fit(X, y)
estimator.support_vectors_[:4]

In [ ]:
estimator.score(X, y)

## Hyperparameter Optimization

There are a few ways to learn the best *hyper*parameters while training. One is `GridSearchCV`.
As the name implies, this does a brute-force search over a grid of hyperparameter combinations.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
%%time
estimator = SVC(gamma='auto', random_state=0, probability=True)
param_grid = {
    'C': [0.001, 10.0],
    'kernel': ['rbf', 'poly'],
}

grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=2)
grid_search.fit(X, y)

## Single-machine parallelism with scikit-learn

![](images/unmerged_grid_search_graph.png)

Scikit-Learn has nice *single-machine* parallelism, via Joblib.
Any scikit-learn estimator that can operate in parallel exposes an `n_jobs` keyword.
This controls the number of CPU cores that will be used.

In [ ]:
%%time
grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=2, n_jobs=-1)
grid_search.fit(X, y)

## Multi-machine parallelism with Dask

![](images/merged_grid_search_graph.png)

Dask can talk to scikit-learn (via joblib) so that your *cluster* is used to train a model. 

If you run this on a laptop, it will take quite some time, but the CPU usage will be satisfyingly near 100% for the duration. To run faster, you would need a distributed cluster. That would mean putting something in the call to `Client` something like

```
c = Client('tcp://my.scheduler.address:8786')
```

Details on the many ways to create a cluster can be found [here](https://docs.dask.org/en/latest/setup/single-distributed.html).

Let's try it on a larger problem (more hyperparameters).

In [ ]:
import joblib
import dask.distributed

c = dask.distributed.Client()

In [ ]:
param_grid = {
    'C': [0.001, 10.0],
    'kernel': ['rbf', 'poly'],
}

In [ ]:
%%time
# TODO

In [ ]:
grid_search.best_params_, grid_search.best_score_

In [ ]:
c.close()

## Dask ML

Dask-ML provides scalable machine learning in Python using Dask alongside popular machine learning libraries like Scikit-Learn, XGBoost, and others.

### Training on large dataset

Most estimators in scikit-learn are designed to work on in-memory arrays. Training with larger datasets may require different algorithms.

Dask ML implements several algorithms that work well on larger than memory datasets, which you might store in a dask array or dataframe.

Let's create a large dataset. In this example, we’ll use dask_ml.datasets.make_blobs to generate some random dask arrays.

In [ ]:
import dask_ml.datasets

X, y = dask_ml.datasets.make_blobs(n_samples=10000000,
                                   chunks=1000000,
                                   random_state=0,
                                   centers=3)
X = X.persist()
X

We’ll use the k-means implemented in Dask-ML to cluster the points. 

In [ ]:
import dask_ml.cluster

In [ ]:
km = dask_ml.cluster.KMeans(n_clusters=3, init_max_iter=2, oversampling_factor=10)
km.fit(X)

Plot a sample of points, colored by the cluster each falls into.

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.scatter(X[::10000, 0], X[::10000, 1], marker='.', c=km.labels_[::10000],
           cmap='viridis', alpha=0.25);

### Perform hyperparameter search with Dask ML

Create a dataset

In [ ]:
from dask_ml.datasets import make_classification
X, y = make_classification(n_samples=100000, n_classes=2, n_redundant=0,
                          random_state=0, shuffle=False,chunks=50)
X = X.persist()
X

In [ ]:
from dask_ml.model_selection import RandomizedSearchCV
from scipy.stats import uniform, loguniform
from sklearn.linear_model import SGDClassifier


model = SGDClassifier(eta0=0.01)
params = {
    "l1_ratio": uniform(0, 1),
    "alpha": loguniform(1e-5, 1e-1),
    "penalty": ["l2", "l1", "elasticnet"],
    "learning_rate": ["invscaling", "adaptive"],
    "power_t": uniform(0, 1),
    "average": [True, False],
}

# TODO

In [ ]:
search.score(X, y)

In [ ]:
search.best_estimator_